# Function Notebook

## Introduction
This notebook is designed to be a flexible and expandable template for developing and documenting functions for various tasks.

## Next Steps 
- Data Reduction ------done
- Feature Selection -----done
- Dimensionality Reduction ------done
- Binning / Encoding-------done
- Feature Engineering-------done
- Text Handling
- Data Validation


## NOTE : This is still in progress 

## Table of Contents1. [Configuration and Setup](#Configuration-and-Setpu)
2. [API Get Dataset no key](#Get-Data-No-ApiKey)
3. [Pre-Processing Functions](#Pre-Processing-Functions)
    - [Dealing with NULL Values (#Finding Missing Data Count )](Dealing-with-NULL-Values-(-Finding-Missing-Data-Count-))
    - [Remove(drop), mean , median , mode](#Remove(drop),-mean-,-median-,-mode)
    - [Data Combining / Intergration](#Data-Combining-/-Intergration)
    - [Normalizing / Feature Scaling](#Normalizing-/-Feature-Scaling)
4. [Everything Maps](#Everything-Maps)
    - [Extracting Coordinates](#Extracting-Coordinates)
    - [Point to point distance calculator minimum](#Point-to-point-distance-calculator-minimum-(-Thomas-))
    - [Point to point distance calculator maximum](#Point-to-point-distance-calculator-maximum)


## Configuration and Setup
Set up the environment with necessary libraries and configurations, Make sure you have all libraries installed under functions 

In [48]:
###################################################################
# Libraries used:
###################################################################
import numpy as np
import pandas as pd
import seaborn as sns
import folium
import matplotlib.pyplot as plt
import requests
import math
import tensorflow as tf
from io import StringIO
from geopy.distance import geodesic
from folium.plugins import MarkerCluster
from sklearn.preprocessing import MinMaxScaler,LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler,PowerTransformer,MaxAbsScaler
from sklearn.preprocessing import RobustScaler,Normalizer,QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from haversine import haversine
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tkinter as tk
from tkinter import messagebox
from sklearn.decomposition import PCA

## Get Data No ApiKey

In [2]:
def API_Unlimited(datasetname): # pass in dataset name and api key
    dataset_id = datasetname

    base_url = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
    #apikey = api_key
    dataset_id = dataset_id
    format = 'csv'

    url = f'{base_url}{dataset_id}/exports/{format}'
    params = {
        'select': '*',
        'limit': -1,  # all records
        'lang': 'en',
        'timezone': 'UTC'
    }

    # GET request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        # StringIO to read the CSV data
        url_content = response.content.decode('utf-8')
        datasetname = pd.read_csv(StringIO(url_content), delimiter=';')
        print(datasetname.sample(10, random_state=999)) # Test
        return datasetname 
    else:
        return (print(f'Request failed with status code {response.status_code}'))


"""
Get unlimited data from the API Function 

Parameters:
datasetname (string): dataset name as from city of melbourn 
apikey (string): the current api Key ( this should be gotton via the below if api stored in current workspace / google drive ( refer to Te API)

f = open("API.txt","r")
api_key = f.read()

Returns:
Csv : Returns the csv dataset of the dataset name 
"""


'\nGet unlimited data from the API Function \n\nParameters:\ndatasetname (string): dataset name as from city of melbourn \napikey (string): the current api Key ( this should be gotton via the below if api stored in current workspace / google drive ( refer to Te API)\n\nf = open("API.txt","r")\napi_key = f.read()\n\nReturns:\nCsv : Returns the csv dataset of the dataset name \n'

#### Testing :

In [ ]:
dataset_id_1 = 'litter-traps'
dataset_id_2 = 'public-barbecues'
dataset_id_3 = 'cafes-and-restaurants-with-seating-capacity'
dataset_id_4 = 'argyle-square-air-quality'
litter_df = API_Unlimited(dataset_id_1)
bbq_df = API_Unlimited(dataset_id_2)
cafe_df = API_Unlimited(dataset_id_3)
AirQuality_df = API_Unlimited(dataset_id_4)

# Pre-Processing Functions

## Dealing with NULL Values ( Finding Missing Data Count )

In [4]:
def FindMissingVal(df):
  #now lets have a array to store the feature with number of NAN values
  MissingFeaturenValues = []
  #now we check each column
  for column in df.columns:
    missingVals = np.sum(df[column].isnull()) # sum the number of NAN values into variable
    MissingFeaturenValues.append({'Feature':column ,'Number of Missing Values':missingVals}) #the array consist of dictionary with feature and its missing values
  return MissingFeaturenValues

"""
Function to get column names with count of missing values 

Parameters:
datasetname (string): dataset name as from city of melbourn 
apikey (string): the current api Key ( this should be gotton via the below if api stored in current workspace / google drive ( refer to Te API)

f = open("API.txt","r")
api_key = f.read()

Returns:
Csv : Returns the csv dataset of the dataset name 
"""

'\nFunction to get column names with count of missing values \n\nParameters:\ndatasetname (string): dataset name as from city of melbourn \napikey (string): the current api Key ( this should be gotton via the below if api stored in current workspace / google drive ( refer to Te API)\n\nf = open("API.txt","r")\napi_key = f.read()\n\nReturns:\nCsv : Returns the csv dataset of the dataset name \n'

In [5]:
FindMissingVal(litter_df)

[{'Feature': 'asset_number', 'Number of Missing Values': 0},
 {'Feature': 'asset_description', 'Number of Missing Values': 0},
 {'Feature': 'construct_material_lupvalue', 'Number of Missing Values': 7},
 {'Feature': 'inspection_frequency', 'Number of Missing Values': 5},
 {'Feature': 'maintained_by', 'Number of Missing Values': 0},
 {'Feature': 'object_type_lupvalue', 'Number of Missing Values': 4},
 {'Feature': 'lat', 'Number of Missing Values': 0},
 {'Feature': 'lon', 'Number of Missing Values': 0},
 {'Feature': 'location', 'Number of Missing Values': 0}]

### Remove(drop), mean , median , mode 

In [6]:


def handle_null_values(dataset, columns, action): # nested conditions
    if action == 'remove':
        modified_dataset = dataset.dropna(subset=columns)
    elif action in ['mean', 'median', 'mode']:
        for column in columns:
            if dataset[column].isnull().any():  
                if action == 'mean':
                    fill_value = dataset[column].mean()
                elif action == 'median':
                    fill_value = dataset[column].median()
                elif action == 'mode':
                    fill_value = dataset[column].mode()[0]
                dataset[column] = dataset[column].fillna(fill_value)
        modified_dataset = dataset
    else:
        raise ValueError("Action must be 'remove', 'mean', 'median', or 'mode'")
    return modified_dataset

"""
Handling Missing Values Functions

Parameters:

dataset(dataframe) -  Dataframe you want to deal null values 
columns (array) - a array of all columns you want to handle missing values for the picked action
actions (string) - 'remove' , 'mode' , 'mean' , 'median' performs the said actions when selected ( can select one at a time )

Returns:
Dataframe : Returns Dataframe including handled values
"""



"\nHandling Missing Values Functions\n\nParameters:\n\ndataset(dataframe) -  Dataframe you want to deal null values \ncolumns (array) - a array of all columns you want to handle missing values for the picked action\nactions (string) - 'remove' , 'mode' , 'mean' , 'median' performs the said actions when selected ( can select one at a time )\n\nReturns:\nDataframe : Returns Dataframe including handled values\n"

#### Testing - I made a array of all columns i want to use mode on and ran function , returns to a new df called modified_mode

In [7]:
"""Usage Example"""
columns=['inspection_frequency','construct_material_lupvalue']
modified_mode = handle_null_values(litter_df,columns,'mode') #<========== Pass DATASET and Prefered Method


In [8]:
FindMissingVal(modified_mode)

[{'Feature': 'asset_number', 'Number of Missing Values': 0},
 {'Feature': 'asset_description', 'Number of Missing Values': 0},
 {'Feature': 'construct_material_lupvalue', 'Number of Missing Values': 0},
 {'Feature': 'inspection_frequency', 'Number of Missing Values': 0},
 {'Feature': 'maintained_by', 'Number of Missing Values': 0},
 {'Feature': 'object_type_lupvalue', 'Number of Missing Values': 4},
 {'Feature': 'lat', 'Number of Missing Values': 0},
 {'Feature': 'lon', 'Number of Missing Values': 0},
 {'Feature': 'location', 'Number of Missing Values': 0}]

### Data Combining / Intergration

In [9]:
import pandas as pd

def Combine_Dataset(datasets, mode='outer'):
    # Check if no datset is given 
    if not datasets:
        raise ValueError("No datasets provided for merging.")
    
    #We check if there are any common columns
    common_columns = set(datasets[0].columns) # making a SET
    for dataset in datasets[1:]:
        common_columns.intersection_update(dataset.columns) #Appending if we find any matching 
        
    #Error if no common found 
    if not common_columns:
        raise ValueError("No common columns available for combining the datasets. Please give datasets with common columns.")

    #Merge
    combined_dataset = datasets[0]
    for dataset in datasets[1:]:
        combined_dataset = pd.merge(combined_dataset, dataset, how=mode, on=list(common_columns))# combine with mode ( default is outer) with the common columns
    
    return combined_dataset

"""
Combining multiple datasets

Parameters:

datasets-  Array of multiple datasets
Mode - inner , outer , left , right    (JOIN) Default : outer

Returns:
Dataframe : Returns Dataframe combined
"""


'\nCombining multiple datasets\n\nParameters:\n\ndatasets-  Array of multiple datasets\nMode - inner , outer , left , right    (JOIN) Default : outer\n\nReturns:\nDataframe : Returns Dataframe combined\n'

#### Testing - I pass a array of all datasets, i want to use mode=inner on and ran the function , returns to a new df called combinedDF

In [10]:
Datasets = [litter_df,cafe_df] # passing 2 datasets As aRRAY HERE 

combinedDF = Combine_Dataset(Datasets, mode='outer') # using mode "outer"

In [11]:
combinedDF.head(5)

,asset_number,asset_description,construct_material_lupvalue,inspection_frequency,maintained_by,object_type_lupvalue,lat,lon,location,census_year,...,building_address,clue_small_area,trading_name,business_address,industry_anzsic4_code,industry_anzsic4_description,seating_type,number_of_seats,longitude,latitude
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-37.77619476202889, 144.93912821315",2013.0,...,52-62 Cade Way PARKVILLE 3052,Parkville,Corner Cafe & Convenience Store,"Shop 1, Ground , 52 Cade Way PARKVILLE 3052",4511.0,Cafes and Restaurants,Seats - Outdoor,6.0,144.939128,-37.776195
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-37.77619476202889, 144.93912821315",2013.0,...,52-62 Cade Way PARKVILLE 3052,Parkville,Corner Cafe & Convenience Store,"Shop 1, Ground , 52 Cade Way PARKVILLE 3052",4511.0,Cafes and Restaurants,Seats - Indoor,35.0,144.939128,-37.776195
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-37.776194762058005, 144.93912821305003",2010.0,...,52-62 Cade Way PARKVILLE 3052,Parkville,Corner Cafe & Convenience Store,"Unit 1, 62-0 Cade Way PARKVILLE 3052",4511.0,Cafes and Restaurants,Seats - Outdoor,6.0,144.939128,-37.776195
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-37.776194762058005, 144.93912821305003",2010.0,...,52-62 Cade Way PARKVILLE 3052,Parkville,Corner Cafe & Convenience Store,"Unit 1, 62-0 Cade Way PARKVILLE 3052",4511.0,Cafes and Restaurants,Seats - Indoor,35.0,144.939128,-37.776195
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-37.776194762090775, 144.93912821290002",2011.0,...,52-62 Cade Way PARKVILLE 3052,Parkville,Corner Cafe & Convenience Store,"Unit 1, 62-0 Cade Way PARKVILLE 3052",4511.0,Cafes and Restaurants,Seats - Indoor,35.0,144.939128,-37.776195


### Normalizing / Feature Scaling 

In [47]:

def Scale_data(dataframe, columns, method='minmax'):

    #Copy so we dont change original dataFrame
    df_scaled = dataframe.copy()
    
    # Check if all specified columns exist in the DataFrame
    if not all(col in df_scaled.columns for col in columns):
        missing_cols = [col for col in columns if col not in df_scaled.columns]
        raise ValueError(f"Columns not found in DataFrame: {missing_cols}")
    
    # Select the normalization method nested if 
    if method == 'minmax':
        scaler = MinMaxScaler()
    elif method == 'zscore':
        scaler = StandardScaler()
    elif method == 'powertransformer':
        scaler = PowerTransformer()
    elif method == 'absscalar':
        scaler = MaxAbsScaler()
    elif method == 'robustscalar':
        scaler = RobustScaler()
    elif method == 'normalizer':
        scaler = Normalizer()
    elif method == 'quantile':
        scaler = QuantileTransformer()
    else:
        raise ValueError("Please Enter one scalar method : minmax , zscore , powertransformer , absscalar , robustscalar , normalizer , quantile") #exception

    # Use the selected scalar
    df_scaled[columns] = scaler.fit_transform(df_scaled[columns])
    
    return df_scaled

"""
Scaling Features in dataset

Parameters:

dataframe-  Array of multiple datasets
columns - array of all columns/features to normalize or scale 
method -  minmax , zscore , powertransformer , absscalar , robustscalar , normalizer , quantile . Default : minmax

Returns:
Dataframe : Returns Dataframe Scaled/Normalized
"""



'\nScaling Features in dataset\n\nParameters:\n\ndataframe-  Array of multiple datasets\ncolumns - array of all columns/features to normalize or scale \nmethod -  minmax , zscore , powertransformer , absscalar , robustscalar , normalizer , quantile . Default : minmax\n\nReturns:\nDataframe : Returns Dataframe Scaled/Normalized\n'

In [13]:
list(AirQuality_df.columns.values)

['time',
 'dev_id',
 'sensor_name',
 'lat_long',
 'averagespl',
 'carbonmonoxide',
 'humidity',
 'ibatt',
 'nitrogendioxide',
 'ozone',
 'particulateserr',
 'particulatesvsn',
 'peakspl',
 'pm1',
 'pm10',
 'pm25',
 'temperature',
 'vbatt',
 'vpanel']

#### Testing : Adding columns I want to scale on AirQuality DF in function , selecting minmax scalar 

In [23]:
 # minmax , zscore , powertransformer , absscalar , robustscalar , normalizer , quantile Are Current Supported 

Scaled_min_max_df = Scale_data(AirQuality_df, ['averagespl','carbonmonoxide','humidity','ibatt','nitrogendioxide','ozone','particulateserr','particulatesvsn','peakspl','pm1','pm10','pm25','temperature'], method='minmax')
Scaled_min_max_df.head(6)

,time,dev_id,sensor_name,lat_long,averagespl,carbonmonoxide,humidity,ibatt,nitrogendioxide,ozone,particulateserr,particulatesvsn,peakspl,pm1,pm10,pm25,temperature,vbatt,vpanel
0,2020-06-09T09:02:38+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.100,0.136969,0.583333,0.866142,0.299700,0.733945,0.0,1.0,0.241379,0.100000,0.075697,0.001002,0.261421,3.96,0.0
1,2020-06-09T11:17:37+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.075,0.115559,0.619048,0.877953,0.322523,0.768807,0.0,1.0,0.120690,0.125000,0.095618,0.001296,0.225888,3.93,0.0
2,2020-06-09T11:32:37+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.075,0.115559,0.630952,0.869423,0.322523,0.768807,0.0,1.0,0.224138,0.158333,0.115538,0.001414,0.215736,3.92,0.0
3,2020-06-09T12:17:37+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.125,0.102704,0.666667,0.874016,0.339940,0.776147,0.0,1.0,0.327586,0.166667,0.115538,0.001709,0.200508,3.91,0.0
4,2020-06-09T13:47:36+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.075,0.115559,0.714286,0.912730,0.333934,0.785321,0.0,1.0,0.120690,0.200000,0.163347,0.002121,0.180203,3.89,0.0
5,2020-06-09T14:02:36+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.125,0.094149,0.738095,0.879921,0.333934,0.785321,0.0,1.0,0.275862,0.133333,0.099602,0.001296,0.182741,3.89,0.0


## Data Reduction PCA

In [43]:

def PCA_Reduction(data, components_num):
    try:
        if not all(data.dtypes.apply(lambda x: pd.api.types.is_numeric_dtype(x))): # error handling , check for numerical values
                raise ValueError("All features must be numerical")
        if data.isnull().values.any():
            raise ValueError("Input data contains NaN values") # if any NAN is in dataset
            
        pca = PCA(n_components=components_num) #setting it up
        
        #Add PCA
        pca_comps = pca.fit_transform(data)
        
        # Create new DF with PCA
        col_names = [f'PC{i+1}' for i in range(components_num)]
        pca_df = pd.DataFrame(data=pca_comps, columns=col_names) # added with num of component names 
        
        # Print dimensions of the new dataset
        print("Dimensions after PCA:", pca_comps.shape)
        
        return pca_df
        
    except ValueError as e:
        print(f"Error: {e}")
        return None, None
    """
    Perform PCA on the given data and return the transformed data PCA along with the new dimensions, depends on PCA components
    
    Parameters:
    data (pd.DataFrame or np.ndarray): The input data for PCA.
    components_num (int): The number of principal components to keep ie :2 ,3 ,4... ( depends on the rank of the matrix covariance)
    
    Returns:
    pd.DataFrame: The transformed data with PCA components as columns.
    """


In [44]:
cols =[
 'averagespl',
 'carbonmonoxide',
 'humidity',
 'ibatt',
 'nitrogendioxide',
 'ozone',
 'particulateserr',
 'particulatesvsn',
 'peakspl',
 'pm1',
 'pm10',
 'pm25',
 'temperature',
 'vbatt',
 'vpanel']
handle_null_values(Scaled_min_max_df,cols,'mean')

,time,dev_id,sensor_name,lat_long,averagespl,carbonmonoxide,humidity,ibatt,nitrogendioxide,ozone,particulateserr,particulatesvsn,peakspl,pm1,pm10,pm25,temperature,vbatt,vpanel
0,2020-06-09T09:02:38+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.100,0.136969,0.583333,0.866142,0.299700,0.733945,0.0,1.0,0.241379,0.100000,0.075697,0.001002,0.261421,3.96,0.00
1,2020-06-09T11:17:37+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.075,0.115559,0.619048,0.877953,0.322523,0.768807,0.0,1.0,0.120690,0.125000,0.095618,0.001296,0.225888,3.93,0.00
2,2020-06-09T11:32:37+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.075,0.115559,0.630952,0.869423,0.322523,0.768807,0.0,1.0,0.224138,0.158333,0.115538,0.001414,0.215736,3.92,0.00
3,2020-06-09T12:17:37+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.125,0.102704,0.666667,0.874016,0.339940,0.776147,0.0,1.0,0.327586,0.166667,0.115538,0.001709,0.200508,3.91,0.00
4,2020-06-09T13:47:36+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.075,0.115559,0.714286,0.912730,0.333934,0.785321,0.0,1.0,0.120690,0.200000,0.163347,0.002121,0.180203,3.89,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141833,2024-05-08T11:37:38+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.075,0.252528,0.595238,0.950787,0.264865,0.675229,0.0,1.0,0.172414,0.216667,0.191235,0.002180,0.489848,3.21,0.00
141834,2024-05-08T05:21:15+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.125,0.376687,0.345238,0.888451,0.207207,0.557798,0.0,1.0,0.172414,0.083333,0.051793,0.000707,0.708122,3.49,12.03
141835,2024-05-08T05:51:15+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.125,0.338167,0.380952,0.729659,0.189790,0.574312,0.0,1.0,0.155172,0.083333,0.047809,0.000707,0.670051,3.56,18.32
141836,2024-05-08T06:51:15+00:00,ems-ec8a,Air Quality Sensor 2,"-37.802772, 144.9655513",0.150,0.291047,0.452381,0.919948,0.230631,0.633028,0.0,1.0,0.189655,0.100000,0.067729,0.001002,0.591371,3.49,1.78


### Testing

In [45]:
to_pcadf = Scaled_min_max_df.drop(columns=['time','dev_id','sensor_name','lat_long'])

pca_df = PCA_Reduction(to_pcadf, 3)

Dimensions after PCA: (141838, 3)


In [46]:
pca_df.head(3)

,PC1,PC2,PC3
0,-11.951030,0.082028,-0.031819
1,-11.952726,0.000637,-0.045359
2,-11.952234,-0.009413,-0.042761


## Encoding

In [55]:
def encode_data(dataset, encoding_type='label', columns=None, target=None):
    if columns is None:
        columns = dataset.columns

    dataset_encoded = dataset.copy()

    if encoding_type == 'label':
        encoder = LabelEncoder()
        for col in columns:
            dataset_encoded[col] = encoder.fit_transform(dataset_encoded[col])

    elif encoding_type == 'one-hot':
        dataset_encoded = pd.get_dummies(dataset_encoded, columns=columns)

    elif encoding_type == 'ordinal':
        encoder = OrdinalEncoder()
        dataset_encoded[columns] = oe.fit_transform(dataset_encoded[columns])

    elif encoding_type == 'target':
        if target is None:
            raise ValueError("Target must be provided for target encoding")
        for col in columns:
            mean_encodings = dataset_encoded.groupby(col)[target].mean()
            dataset_encoded[col] = dataset_encoded[col].map(mean_encodings)
    else:
        raise ValueError(f"Unsupported encoding type: {encoding_type}")

    return dataset_encoded

"""
    Encodes categorical features in a DataFrame using the specified encoding type.

    Parameters:
    - X: pd.DataFrame
        The input data containing features to be encoded.
    - encoding_type: str
        The type of encoding to use. Options are 'label', 'one-hot', 'ordinal', 'target'.
    - columns: list of str (optional)
        List of column names to be encoded. If None, all columns are encoded.
    - target: pd.Series (optional)
        The target variable used for target encoding.

    Returns:
    - X_encoded: pd.DataFrame
        DataFrame with encoded features.
"""

# Example usage:
# df = pd.DataFrame({'Feature1': ['A', 'B', 'A', 'C'],
#                    'Feature2': ['X', 'Y', 'X', 'Z']})
# encoded_df = encode_data(df, encoding_type='one-hot', columns=['Feature1'])

"\n    Encodes categorical features in a DataFrame using the specified encoding type.\n\n    Parameters:\n    - X: pd.DataFrame\n        The input data containing features to be encoded.\n    - encoding_type: str\n        The type of encoding to use. Options are 'label', 'one-hot', 'ordinal', 'target'.\n    - columns: list of str (optional)\n        List of column names to be encoded. If None, all columns are encoded.\n    - target: pd.Series (optional)\n        The target variable used for target encoding.\n\n    Returns:\n    - X_encoded: pd.DataFrame\n        DataFrame with encoded features.\n"

### Testing


## Feature Engineering

## Data Validation

# Everything Maps

### Extracting Coordinates 

In [15]:

def ExtractCoords(df, lat_col=None, long_col=None, combined_col=None):
    coordinates = [] # Array of coordinates initilize 
    try:
        if combined_col: # if the coordinate column is coombined 
            coordinates = [(float(str(coord).split(',')[0].strip()), float(str(coord).split(',')[1].strip()))
                           for coord in df[combined_col] if ',' in str(coord)] # pass the coordinate ( if combined )
        elif lat_col and long_col: # if there is seperate lat and long we pass 
            coordinates = [(float(df.at[i, lat_col]), float(df.at[i, long_col]))
                           for i in df.index]
        else:
            # Auto-detect columns based on common naming conventions
            for col in df.columns:
                if 'lat' in col.lower() and not lat_col: # try this if above not work 
                    lat_col = col
                elif ('lon' in col.lower() or 'lng' in col.lower()) and not long_col:
                    long_col = col
            if lat_col and long_col: # pass regardless of the naming 
                coordinates = [(float(df.at[i, lat_col]), float(df.at[i, long_col]))
                               for i in df.index]
            else:
                raise ValueError("Appropriate coordinate columns not provided or found.")
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

    return coordinates
    
"""
Extract coordinates as tuples (latitude, longitude)

Parameters:

dataframe-  Single Dataframe 
lat_col - Name of Latitude Column
long_col - Name of longitude Column
combined_col - Name of Combined Coordinate column ( if data point for column is [ 123.0  123.5 ] )


Returns:
Dataframe : Array of coords
"""

# Extract coordinates as tuples (latitude, longitude)
#litter_coords = list(zip(litter_df['lat'], litter_df['lon'])) 
#bbq_coords = [(float(c.split(',')[0]), float(c.split(',')[1])) for c in bbq_df['Co-ordinates']]
#cafe_coords = list(zip(cafe_df['Latitude'], cafe_df['Longitude']))

#### Testing : Calling 3 types of coordiante extraction

For this first one , we use the seperate column names with the coordinates and put None for combined coordinate column

In [16]:
litter_coords = ExtractCoords(litter_df,"lat","lon",None)

In [19]:
litter_coords[1:10]

[(-37.812671357856935, 144.94563578253997),
 (-37.825490336480286, 144.9612080150195),
 (-37.81990017409475, 144.94275387398454),
 (-37.81534931155489, 144.94534809647914),
 (-37.819071778292574, 144.9470731818341),
 (-37.82246168061252, 144.93808734493956),
 (-37.82407188705236, 144.94844051872718),
 (-37.83028055230784, 144.96592974982084),
 (-37.80200250662716, 144.9367682878648)]

For this second one , we use the combined column named co_ordinates as in the dataset this column has the combined coordinates lat and lon

In [25]:
bbq_coords = ExtractCoords(bbq_df,None,None,'co_ordinates')

In [26]:
bbq_coords[1:10]

[(-37.839163364409444, 144.98066702168143),
 (-37.78998719902451, 144.94502340388996),
 (-37.786978549708806, 144.95295755838),
 (-37.79622323577933, 144.9266473544793),
 (-37.82009381572264, 144.97510314880833),
 (-37.823064959466265, 144.94180279898958),
 (-37.79744246261081, 144.9233706625667),
 (-37.842387153356434, 144.98190820378719),
 (-37.82744680370206, 144.981675215773)]

For this third one , we use the Name column named latitude , longitude as in the dataset this column has the seperate coordinates lat and lon

In [28]:
cafe_coords = ExtractCoords(cafe_df,"latitude","longitude",None) 

In [30]:
cafe_coords[1:10]

[(-37.817777826050005, 144.96994164279243),
 (-37.8198754458, 144.95648638781466),
 (-37.8198754458, 144.95648638781466),
 (-37.82059511593975, 144.9556348088),
 (-37.82039904747718, 144.95596914650002),
 (-37.82016599807918, 144.9571504476),
 (-37.820200051650005, 144.95510422847929),
 (-37.82001949059274, 144.95583815835),
 (-37.81936815835749, 144.95810006565)]

## Point to point distance calculator minimum ( Thomas )


In [31]:
# Function to calculate the minimum distance from a point to any point in a list
"""
Calculate the minimum geodesic distance from a point to any point in a given list.

Parameters:
point (tuple): A tuple representing the coordinates (latitude, longitude) of the point.
list_of_points (list of tuples): A list of tuples, each representing coordinates (latitude, longitude) of points to compare against.

Returns:
float: The minimum Euclidean distance from the given point to the closest point in the list.
"""

def min_distance(point, list_of_points): 
    return min([geodesic(point, pt).meters for pt in list_of_points]) #get min dis

#example :


row = {'lat': 40.7128, 'lon': -74.0060}
# Call the lambda function with the row as an argument
value = lambda row: min_distance((row['lat'], row['lon']), bbq_coords)
# Get the result by calling the lambda function
result = value(row)
# Print the result
print("test distance in meters :",result)

# example used in dataset :


litter_df['Nearest BBQ Distance (m)'] = litter_df.apply(lambda row: min_distance((row['lat'], row['lon']), bbq_coords), axis=1)
#creates a new column for nearest distance to a point

test distance in meters : 16669936.382948814


## Point to point distance calculator maximum 


In [34]:
# Function to calculate the maximum distance from a point to any point in a list
"""
Calculate the maximum geodesic distance from a point to any point in a given list.

Parameters:
point (tuple): A tuple representing the coordinates (latitude, longitude) of the point.
list_of_points (list of tuples): A list of tuples, each representing coordinates (latitude, longitude) of points to compare against.

Returns:
float: The maximum Euclidean distance from the given point to the closest point in the list.
"""

def max_distance(point, list_of_points): 
    return max([geodesic(point, pt).meters for pt in list_of_points]) #get min dis

#example : 

value = max_distance((row['lat'], row['lon']), bbq_coords) 

In [35]:
value

16674952.993879572

## Number of points in a given radius 

In [67]:
#Calculate the Number of points in a radius from a point 
"""
Calculate the number of geodesic distances from a point to any point in a given list.

Parameters:
center_point (tuple): A tuple representing the coordinates (latitude, longitude) of the point.
list_of_points (list of tuples): A list of tuples, each representing coordinates (latitude, longitude) of points to compare against.
radius_meters - radius in meters

Returns:
INT: The Number of points in the radius given
"""

def count_points_in_radius(center_point, list_of_points, radius_meters):
    count = 0
    for point in list_of_points:
        if geodesic(point, center_point).m <= radius_meters:
            count += 1
    return count



radius = 10000 # in meters 
center_point = (-37.819071778292574, 144.9470731818341) # centure coordiane of my choice 
values = count_points_in_radius(center_point,litter_coords,radius) # count of coordijnates

In [68]:
values

63

## The Map using folium ( basic )


In [69]:
from folium.plugins import MarkerCluster

    
"""
Calculate the minimum geodesic distance from a point to any point in a given list.

Parameters:
dataframe : A datset representing the coordinates (latitude, longitude) of the index and also other values hence when
using this we can also include other things from the dataset in the map , when using the html legend

Returns:
Map: The folium based map is returned
"""

def map_func(PointsDatasets,):
    # Create a folium map centered at the mean coordinates of litter traps / intial setup
    map_center = [PointsDatasets['lat'].mean(), PointsDatasets['lon'].mean()]
    mymap = folium.Map(location=map_center, zoom_start=13)
    
    # Add circles for the points
    for index, row in PointsDatasets.iterrows():
        location = [row['lat'], row['lon']] 
        # Add a circle for the radius around the litter trap
        folium.Circle(
            location=location,
            radius=30,
            color='red',
            fill=True,
            fill_opacity=0.2
        ).add_to(mymap)
    return mymap

# Example usage ========================= Pass in your function =================
"""Make sure your dataframe has a column with both lat and lon"""
map_func(litter_df)

## Corellation Maps

### correlation heat map for spearman and pearson correlation

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

def plot_correlation_heatmaps(data, labels, order=None):
    """
    Plots Pearson and Spearman correlation heatmaps for the given data.

    Parameters:
    - data: 2D numpy array or DataFrame containing the data to analyze.
    - labels: List of column names corresponding to the data.
    - order: List of indices specifying the order of columns for aesthetic purposes in the heatmap.

    The function creates a figure with two subplots: one for Pearson correlation and one for Spearman correlation.
    """
    if order is None:
        order = range(len(labels))  # Default order if none provided

    # Compute Pearson correlation coefficients
    R = np.corrcoef(data, rowvar=False)

    # Compute Spearman's rank correlation
    rho, pval = spearmanr(data, axis=0)

    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot Pearson correlation heatmap
    ax1.set_title('Pearson Correlation')
    plt.sca(ax1)
    corrheatmap(R[np.ix_(order, order)], np.array(labels)[order])

    # Plot Spearman correlation heatmap
    ax2.set_title('Spearman Correlation')
    plt.sca(ax2)
    corrheatmap(rho[np.ix_(order, order)], np.array(labels)[order])

    plt.show()

def corrheatmap(R, labels):
    """
    Helper function to draw a correlation heat map.
    """
    k = len(labels)
    plt.imshow(R, cmap='RdBu', vmin=-1, vmax=1)
    plt.xticks(np.arange(k), labels=labels, rotation=45)
    plt.yticks(np.arange(k), labels=labels)
    plt.colorbar()
    for i in range(k):
        for j in range(k):
            plt.text(j, i, f"{R[i, j]:.2f}", ha="center", va="center",
                     color="white" if np.abs(R[i, j]) > 0.5 else "black")
    plt.grid(False)

# Usage example
# data = np.random.rand(100, 5)  # Dummy data
# labels = ['Var1', 'Var2', 'Var3', 'Var4', 'Var5']
# order = [0, 1, 2, 3, 4]
# plot_correlation_heatmaps(data, labels, order)


In [23]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

def optimal_k_clusters(data, k_range):
    """
    Determines the optimal number of clusters for K-means clustering based on silhouette scores.

    Parameters:
    - data: The dataset on which clustering is to be performed.
    - k_range: A range of k values to test. Typically, this is a range object.

    Returns:
    - optimal_k: The optimal number of clusters with the highest silhouette score.
    - Plots the silhouette scores for each k in k_range.
    """
    # List to store silhouette scores for each value of k
    silh_scores = []

    # Iterate over each value of k in the range provided
    for k in k_range:
        # Fit KMeans clustering model to the data with 'k' clusters
        kmeans = KMeans(n_clusters=k, n_init=10) # n_init=10 to ensure consistency across initializations
        cluster_labels = kmeans.fit_predict(data)
        
        # Calculate the silhouette score for the current number of clusters
        silhouette_avg = silhouette_score(data, cluster_labels)
        silh_scores.append(silhouette_avg)

    # Determine the value of k that has the maximum silhouette score
    optimal_k = k_range[np.argmax(silh_scores)]
    print("Optimal number of clusters (k):", optimal_k)

    # Plot the silhouette scores against the number of clusters
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, silh_scores, marker='o')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score for Different Values of k')
    plt.grid(True)
    plt.show()

    return optimal_k

# Example of how to use the function
# data = your_data_frame  # make sure to define your DataFrame
# k_range = range(2, 11)  # Setting a range from 2 to 10
# optimal_k = optimal_k_clusters(data, k_range)


In [24]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

def find_optimal_clusters(data, k_range=(2, 12)):
    """
    Determines the optimal number of clusters for K-means clustering using the elbow method and plots the results.

    Parameters:
    - data: The dataset on which clustering is to be performed, typically preprocessed (e.g., PCA-transformed).
    - k_range: A tuple indicating the range of k values to test (inclusive). Default is (2, 12).

    Returns:
    - Plots the elbow plot showing the distortion for each k, helping to identify the optimal number of clusters.
    """
    # Initialize the KMeans model with a fixed number of initializations to avoid random seed variability
    model = KMeans(n_init=10)

    # Initialize the KElbowVisualizer with the KMeans model, specifying the range of k and the metric 'distortion'
    visualizer = KElbowVisualizer(
        model, k=k_range, metric='distortion', timings=False
    )

    # Fit the visualizer to the data
    visualizer.fit(data)

    # Finalize and render the figure
    visualizer.show()

# Example of how to use the function
# X_pca = your_pca_transformed_data  # Ensure your data is appropriately preprocessed, e.g., using PCA
# find_optimal_clusters(X_pca)


#### pca and TSNE function


In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np

def dimensionality_reduction(X, method='pca', n_components=2, **kwargs):
    """
    Applies PCA or t-SNE to reduce the dimensionality of the input data.

    Parameters:
    - X: np.array or pd.DataFrame
        The input data to be reduced.
    - method: str
        The method to use for dimensionality reduction. Options are 'pca' or 'tsne'.
    - n_components: int
        The number of dimensions to reduce to. Default is 2.
    - kwargs: dict
        Additional arguments to be passed to the selected method.

    Returns:
    - X_reduced: np.array
        The reduced dataset.
    """
    if method == 'pca':
        reducer = PCA(n_components=n_components, **kwargs)
    elif method == 'tsne':
        reducer = TSNE(n_components=n_components, **kwargs)
    else:
        raise ValueError(f"Unsupported reduction method: {method}")

    X_reduced = reducer.fit_transform(X)
    return X_reduced

# Example usage:
# from sklearn.datasets import load_iris
# data = load_iris()
# X_reduced = dimensionality_reduction(data.data, method='pca', n_components=2)
# X_reduced_tsne = dimensionality_reduction(data.data, method='tsne', n_components=2)


#### (Label Encoding: One-Hot Encoding: Ordinal Encoding: Target Encoding:) functions

In [ ]:


def encode_data(X, encoding_type='label', columns=None, target=None):
    """
    Encodes categorical features in a DataFrame using the specified encoding type.

    Parameters:
    - X: pd.DataFrame
        The input data containing features to be encoded.
    - encoding_type: str
        The type of encoding to use. Options are 'label', 'one-hot', 'ordinal', 'target'.
    - columns: list of str (optional)
        List of column names to be encoded. If None, all columns are encoded.
    - target: pd.Series (optional)
        The target variable used for target encoding.

    Returns:
    - X_encoded: pd.DataFrame
        DataFrame with encoded features.
    """
    if columns is None:
        columns = X.columns

    X_encoded = X.copy()

    if encoding_type == 'label':
        le = LabelEncoder()
        for col in columns:
            X_encoded[col] = le.fit_transform(X_encoded[col])

    elif encoding_type == 'one-hot':
        X_encoded = pd.get_dummies(X_encoded, columns=columns)

    elif encoding_type == 'ordinal':
        oe = OrdinalEncoder()
        X_encoded[columns] = oe.fit_transform(X_encoded[columns])

    elif encoding_type == 'target':
        if target is None:
            raise ValueError("Target must be provided for target encoding")
        for col in columns:
            mean_encodings = X_encoded.groupby(col)[target].mean()
            X_encoded[col] = X_encoded[col].map(mean_encodings)
    else:
        raise ValueError(f"Unsupported encoding type: {encoding_type}")

    return X_encoded

# Example usage:
# df = pd.DataFrame({'Feature1': ['A', 'B', 'A', 'C'],
#                    'Feature2': ['X', 'Y', 'X', 'Z']})
# encoded_df = encode_data(df, encoding_type='one-hot', columns=['Feature1'])


In [ ]:
#### DEcision Boundary for KNN classifier

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

def plot_estimator(estimator, X, y, mesh_step=0.1, cmap_light=None, cmap_bold=None):
    """
    Plots the decision boundaries of an estimator alongside the data points.

    Parameters:
    - estimator: object
        The model that follows scikit-learn API, should have a .fit() and .predict() method.
    - X: np.array
        The features data (2D array).
    - y: np.array
        The target labels.
    - mesh_step: float (optional)
        The step size for mesh grid. Default is 0.1.
    - cmap_light: ListedColormap (optional)
        The colormap for mesh background. Default is a preset colormap.
    - cmap_bold: ListedColormap (optional)
        The colormap for scatter plot. Default is a preset colormap.

    Returns:
    - None
        Displays the decision boundary plot.
    """
    # Set default colormaps if not provided
    if cmap_light is None:
        cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
    if cmap_bold is None:
        cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

    estimator.fit(X, y)
    
    # Determine the maximum and minimum mesh as a boundary
    x_min, x_max = X[:, 0].min() - mesh_step, X[:, 0].max() + mesh_step
    y_min, y_max = X[:, 1].min() - mesh_step, X[:, 1].max() + mesh_step

    # Generate points on the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, mesh_step),
                         np.arange(y_min, y_max, mesh_step))

    # Make predictions on the grid points
    Z = estimator.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot the decision boundary
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot the original data points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title('Decision Boundary Plot')
    plt.show()

#plot_estimator(your_knn_model, your_Xtrain, your_ytrain)